# Getting the info that the client needs for D3.js

Often it is easier to transform data in the backend to be used by D3. In general all data in D3 needs to be a list of dicts. In this example I'm using an existing `world` object to demonstrate the methods that do these transformations. 

In [1]:
#I'm mapping to the actual files in the repo so that I can also use this to troubleshoot
import os
import sys
import numpy as np
import pandas as pd
import pickle
import django
sys.path.append('../..')
#Loading my project settings from prodweb. This allows me to load and query models. 
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'prodweb.settings')
django.setup()


with (open("../pickles/billmanhworld.pkl", 'rb')) as pickle_file:
    world = pickle.load(pickle_file)

In [2]:
sys.path.append('..')
from lib.builders import world as w

`w.get_area_data(world)` gets the area relevant to where the character is. This includes surrounding areas.

In [3]:
world

In [4]:
print(world.Character.location)
pd.DataFrame(w.get_area_data(world)).T

[19, 12]


,aware,danger,elevation,feature,key,nation,nation number,rainfall,terrain,turn_last_visited,visited,x,y
area,1,1.294,11,none,19:12,Starbrook,6,0,desert,0,0,19,12
NArea,1,0.981,10,none,19:11,Starbrook,6,0,desert,33,1,19,11
SArea,0,0.692,13,none,19:13,Ravenmain,0,0,desert,0,0,19,13
EArea,1,1.045,5,none,20:12,none,none,0,ocean,39,1,20,12
WArea,1,0.581,17,none,18:12,Starbrook,6,0,desert,0,0,18,12


For other game principals, it's easy to just make regular `pandas` and `numpy` queries on those datasets. 

In [5]:
world.df_features.loc[world.Character.get_location_key(),'visited'] >= 1

False

Querying the terrain is easy as well. For example, here is a query of all of the world where the _terrain_ is _desert_ and the _danger_ is above .5. These queries are fast and cheap. 

In [6]:
world.df_features.loc[(world.df_features['terrain']=='desert') & 
                     (world.df_features['danger']>=.5)].head()

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
6:4,4,0.0,6,6:4,8.0,desert,NaN,3.0,Warchild,0,0,0,1.210
7:4,4,0.0,7,7:4,7.0,desert,NaN,3.0,Warchild,0,0,0,0.933
8:4,4,0.0,8,8:4,6.0,desert,NaN,3.0,Warchild,0,0,0,0.731
9:4,4,0.0,9,9:4,7.0,desert,NaN,3.0,Warchild,0,0,0,0.827
12:4,4,0.0,12,12:4,6.0,desert,NaN,6.0,Starbrook,0,1,0,1.156


You can also _mask_ the terrain where the character has no knowledge. 

In [7]:
masked_world = w.mask_unknown(world)
world.Character.title = "Noble"
masked_world.head()

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
0:0,0,0.0,0,0:0,2.0,ocean,NaN,NaN,NaN,0,1,0,0.500
1:0,0,0.0,1,1:0,-2.0,ocean,NaN,NaN,NaN,0,1,0,0.882
2:0,0,0.0,2,2:0,1.0,ocean,NaN,NaN,NaN,0,1,0,0.427
3:0,0,0.0,3,3:0,1.0,ocean,NaN,NaN,NaN,0,1,0,0.467
4:0,0,0.0,4,4:0,1.0,ocean,NaN,NaN,NaN,0,1,0,0.759


Querying people is also easy, but you need to use _list comprehension_ to do it. This is the general structure. 

In [8]:
[[person.name for person in t.population] for t in world.towns]

[['Monkam', 'Quisassif', 'Xayangoh'],
 ['Pirnab', 'Chirnorb'],
 ['Hallaf', 'Iragordis', 'Olordave'],
 ['Trirnaf', 'Tickich', 'Drornorb', 'Glasasselle', 'Scomattoh'],
 ['Alrpad', 'Heklich'],
 ['Pinoddof', 'Aniladdich', 'Deggid', 'Quosassace'],
 ['Ekoxeb'],
 ['Scoscadle', 'Rogennaf', 'Fewaker'],
 ['Vockrich', 'Milarkunt'],
 ['Theladdottle'],
 ['Scafillion', 'Shelandai', 'Rascundle', 'Thellut'],
 ['Stifarrunt', 'Quappraf'],
 ['Rekillud', 'Ulckow', 'Legsid'],
 ['Pozed', 'Helarkel'],
 ['Nocinnaker'],
 ['Nekkis', 'Fapprif'],
 ['Stefillow'],
 ['Erokillid'],
 ['Urnnadle'],
 ['Pollox', 'Jomendish'],
 ['Pogordeb'],
 ['Grorpun', 'Ultendace', 'Detoldion'],
 ['Bolanda'],
 ['Thalandep', 'Maharkea'],
 ['Laggnan']]

Query the people in the town by setting conditions in the list. These people are the highest in temperment. 

In [9]:
[[(person.name,person.temperment) for person in t.population if person.temperment>.5] for t in world.towns]

[[],
 [],
 [('Hallaf', 0.9), ('Olordave', 0.83)],
 [('Trirnaf', 0.94), ('Tickich', 0.6), ('Scomattoh', 0.67)],
 [('Alrpad', 0.8)],
 [('Pinoddof', 0.75), ('Deggid', 0.91)],
 [('Ekoxeb', 0.93)],
 [('Scoscadle', 0.64), ('Rogennaf', 0.78)],
 [('Milarkunt', 0.83)],
 [],
 [('Rascundle', 0.65), ('Thellut', 0.98)],
 [('Quappraf', 0.69)],
 [('Ulckow', 0.58), ('Legsid', 0.75)],
 [],
 [],
 [('Nekkis', 0.87), ('Fapprif', 0.97)],
 [('Stefillow', 0.53)],
 [],
 [('Urnnadle', 0.98)],
 [('Pollox', 0.61)],
 [('Pogordeb', 0.61)],
 [('Grorpun', 0.64), ('Ultendace', 0.67), ('Detoldion', 0.98)],
 [('Bolanda', 0.94)],
 [('Thalandep', 0.76), ('Maharkea', 0.51)],
 [('Laggnan', 0.96)]]

You can also get summary stats on people. For example, the average temperment per town.

In [10]:
[(t.name,np.mean([person.temperment for person in t.population])) for t in world.towns]

[('Corerun', 0.3233333333333333),
 ('Hellcombe', 0.15000000000000002),
 ('Hardspring', 0.67),
 ('Earthrise', 0.6),
 ('Rockfort', 0.505),
 ('Kingsfellow', 0.46499999999999997),
 ('Stenchcraft', 0.93),
 ('Puregarden', 0.5),
 ('Hellspring', 0.42),
 ('Kingsloch', 0.26),
 ('Snowborourgh', 0.5525),
 ('Soilpike', 0.5249999999999999),
 ('Rageplace', 0.59),
 ('Lionhenge', 0.37),
 ('Heirwish', 0.23),
 ('Heirhenge', 0.9199999999999999),
 ('Starhenge', 0.53),
 ('Demonscrest', 0.33),
 ('Coretear', 0.98),
 ('Splitcombe', 0.47),
 ('Ratfort', 0.61),
 ('Lilyspring', 0.7633333333333333),
 ('Buckfold', 0.94),
 ('Warrun', 0.635),
 ('Flameforth', 0.96)]

For some purposes you may want to get a collection of people and edit them in bulk. This is also possible using list comprehension. 

In [11]:
where_the_char_has_been = world.df_features.loc[world.df_features['visited']==1]
where_the_char_has_been

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
13:7,7,0.0,13,13:7,32.0,town,Kingsloch,6.0,Starbrook,1,1,9,0.804
14:7,7,0.0,14,14:7,23.0,desert,NaN,6.0,Starbrook,1,1,10,1.029
13:8,8,0.0,13,13:8,37.0,mountain,NaN,6.0,Starbrook,1,1,8,0.952
14:8,8,0.0,14,14:8,25.0,mountain,NaN,6.0,Starbrook,1,1,17,0.687
13:9,9,0.0,13,13:9,44.0,mountain,NaN,6.0,Starbrook,1,1,7,0.841
14:9,9,0.0,14,14:9,30.0,mountain,NaN,6.0,Starbrook,1,1,18,1.048
15:9,9,0.0,15,15:9,19.0,desert,NaN,6.0,Starbrook,1,1,21,0.780
16:9,9,0.0,16,16:9,15.0,desert,NaN,6.0,Starbrook,1,1,22,0.766
17:9,9,1.0,17,17:9,12.0,desert,NaN,6.0,Starbrook,1,1,23,0.711


In [12]:
where_the_char_has_been['nation'].dropna()

key
13:7     Starbrook
14:7     Starbrook
13:8     Starbrook
14:8     Starbrook
13:9     Starbrook
14:9     Starbrook
15:9     Starbrook
16:9     Starbrook
17:9     Starbrook
18:9     Starbrook
12:10    Starbrook
13:10    Starbrook
14:10    Starbrook
15:10    Starbrook
17:10    Starbrook
18:10    Starbrook
17:11    Starbrook
18:11    Starbrook
19:11    Starbrook
20:11    Starbrook
17:12    Starbrook
Name: nation, dtype: object

In [13]:
nations_where_character_has_been = np.unique(where_the_char_has_been['nation'].dropna())
nations_where_character_has_been

array(['Starbrook'], dtype=object)

You can use this to get lists of people who the character knows. 

In [14]:
[[t for t in n.towns] for n in world.nations if n.name in nations_where_character_has_been]

[[town of Kingsloch: population: 1 location: [13,7] founded 3,
  capitol of Rageplace: population: 2 location: [12,10] founded 4,
  town of Ratfort: population: 1 location: [17,10] founded 8]]

In [15]:
[[t for t in T.population] for T in world.towns if T.nation in nations_where_character_has_been]

[[Theladdottle the Speaker of Kingsloch],
 [Rekillud the Speaker of Rageplace,
  Ulckow the commoner,
  Legsid the Ruler of the nation of Starbrook],
 [Pogordeb the Speaker of Ratfort]]

In [16]:
[[t.get_person_data() for t in T.population] for T in world.towns if T.nation in nations_where_character_has_been]

[[{'name': 'Theladdottle',
   'role': 'Speaker of Kingsloch',
   'loyalty': 0.5,
   'temperment': 0.26,
   'attributes': ['alive'],
   'messages': ['Hello stranger.',
    'None',
    'None',
    'What a great wedding. So glad the people of [Nation of Starbrook] are our friends.']}],
 [{'name': 'Rekillud',
   'role': 'Speaker of Rageplace',
   'loyalty': 0.5,
   'temperment': 0.44,
   'attributes': ['alive'],
   'messages': ['Hello stranger.',
    'None',
    'None',
    'What a great wedding. So glad the people of [Nation of Starbrook] are our friends.']},
  {'name': 'Ulckow',
   'role': 'commoner',
   'loyalty': 0.5,
   'temperment': 0.58,
   'attributes': ['alive'],
   'messages': ['Hello stranger.',
    'None',
    'None',
    'What a great wedding. So glad the people of [Nation of Starbrook] are our friends.']},
  {'name': 'Legsid',
   'role': 'Ruler of the nation of Starbrook',
   'loyalty': 0.5,
   'temperment': 0.75,
   'attributes': ['alive'],
   'messages': ['Hello stranger.',

In [17]:
masked_world.loc[world.Character.get_location_key()]

y                           12
rainfall                     0
x                           19
key                      19:12
elevation                   11
terrain                 desert
feature                    NaN
nation number                6
nation               Starbrook
visited                      0
aware                        1
turn_last_visited            0
danger                   1.294
Name: 19:12, dtype: object

You can query nations or towns. `world.nations` is a list of _nations_ and `world.towns` is a list of _towns_

In [18]:
where_the_char_has_been = world.df_features.loc[(world.df_features['visited']==1)&
                                                (world.df_features['terrain']=='town')]
where_the_char_has_been

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
13:7,7,0.0,13,13:7,32.0,town,Kingsloch,6.0,Starbrook,1,1,9,0.804
12:10,10,0.0,12,12:10,62.0,town,Rageplace,6.0,Starbrook,1,1,5,0.604
17:10,10,0.0,17,17:10,19.0,town,Ratfort,6.0,Starbrook,1,1,27,0.402


## Getting the diplomacy and relationships data for the journal page

In [19]:
rel = w.get_people_where_char_has_visited(world)
rel

[{'town': town of Kingsloch: population: 1 location: [13,7] founded 3,
  'people': [Theladdottle the Speaker of Kingsloch]},
 {'town': capitol of Rageplace: population: 2 location: [12,10] founded 4,
  'people': [Rekillud the Speaker of Rageplace,
   Ulckow the commoner,
   Legsid the Ruler of the nation of Starbrook]},
 {'town': town of Ratfort: population: 1 location: [17,10] founded 8,
  'people': [Pogordeb the Speaker of Ratfort]}]

In [49]:
w.get_relationships_node_map(world)

[{'name': 'Kingsloch',
  'children': [{'name': 'Theladdottle',
    'role': 'Speaker of Kingsloch',
    'temperment': 0.26,
    'loyalty': 0.5}]},
 {'name': 'Rageplace',
  'children': [{'name': 'Rekillud',
    'role': 'Speaker of Rageplace',
    'temperment': 0.44,
    'loyalty': 0.5},
   {'name': 'Ulckow', 'role': 'commoner', 'temperment': 0.58, 'loyalty': 0.5},
   {'name': 'Legsid',
    'role': 'Ruler of the nation of Starbrook',
    'temperment': 0.75,
    'loyalty': 0.5}]},
 {'name': 'Ratfort',
  'children': [{'name': 'Pogordeb',
    'role': 'Speaker of Ratfort',
    'temperment': 0.61,
    'loyalty': 0.5}]}]

In [56]:
rel[0]['town'].key

'13:7'

In [66]:
p = rel[0]['town'].population[0]


In [67]:
p.

['alive']

In [68]:
def get_relationships_nodes_and_links(world):
    nodes = [{'name':r['town'].name,
              'title':str(r['town']).split(":")[0],
              'nation':r['town'].diplomacy['nation'],
              'population':r['town'].pop,
              'type':r['town'].type,
              'location':r['town'].key,
              'founded year':r['town'].founded,
              'national fealty':r['town'].diplomacy['national fealty'],
             'children': [{'name':p.name,
                          'role':p.role,
                          'temperment':p.temperment,
                          'loyalty':p.loyalty,
                          'type':'person'} for p in r['people']]} 
             for r in w.get_people_where_char_has_visited(world)]
    return nodes

get_relationships_nodes_and_links(world)

[{'name': 'Kingsloch',
  'title': 'town of Kingsloch',
  'nation': 'Starbrook',
  'population': 1,
  'type': 'town',
  'location': '13:7',
  'founded year': 3,
  'national fealty': 1,
  'children': [{'name': 'Theladdottle',
    'role': 'Speaker of Kingsloch',
    'temperment': 0.26,
    'loyalty': 0.5,
    'type': 'person'}]},
 {'name': 'Rageplace',
  'title': 'capitol of Rageplace',
  'nation': 'Starbrook',
  'population': 2,
  'type': 'capitol',
  'location': '12:10',
  'founded year': 4,
  'national fealty': 1,
  'children': [{'name': 'Rekillud',
    'role': 'Speaker of Rageplace',
    'temperment': 0.44,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Ulckow',
    'role': 'commoner',
    'temperment': 0.58,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Legsid',
    'role': 'Ruler of the nation of Starbrook',
    'temperment': 0.75,
    'loyalty': 0.5,
    'type': 'person'}]},
 {'name': 'Ratfort',
  'title': 'town of Ratfort',
  'nation': 'Starbrook',
  'population